In [9]:
import os
import numpy as np
import generate_HNF
import spglib
from copy import deepcopy
import permutation
import generate_superlattice
import unko
from pymatgen.core import Lattice, Structure
from pymatgen.core.periodic_table import Specie, DummySpecie
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.cif import CifWriter

rutile = ([(4, 0, 0),

           (0, 4, 0),

           (0, 0, 3)],

          [(0, 0, 0),

           (0.5, 0.5, 0.5),

           (0.3, 0.3, 0.0),

           (0.7, 0.7, 0.0),

           (0.2, 0.8, 0.5),

           (0.8, 0.2, 0.5)],

          [14, 14, 8, 8, 8, 8])
o_sublattice = ([(4, 0, 0),

                 (0, 4, 0),

                 (0, 0, 3)],

                [(0.3, 0.3, 0.0),

                 (0.7, 0.7, 0.0),

                 (0.2, 0.8, 0.5),

                 (0.8, 0.2, 0.5)],

                [8, 8, 8, 8])
parent_sym = spglib.get_symmetry(rutile)
HNF_list = generate_HNF.generate_all_superlattices(2)
reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, parent_sym["rotations"])

In [10]:

def write_cif(filename, struct, refine_cell=False, resize_volume=False):

    struct = refine_and_resize_structure(struct, refine_cell, resize_volume)

    if not struct.is_valid(1e-4):

        return

    cw = CifWriter(struct)

    cw.write_file(filename)





def refine_and_resize_structure(struct, refine_cell=True, resize_volume=True):


    if refine_cell:

        sga = SpacegroupAnalyzer(struct, symprec=1e-6, angle_tolerance=1e-2)

        struct = sga.get_primitive_standard_structure()



    return struct

In [11]:
parent_lattice = unko.get_superlattice(rutile, reduced_HNF[4], 2)
o_sublattice = unko.get_o_superlattice(rutile, reduced_HNF[4], 2)
parent_lattice

([[4, 4, 0], [0, 8, 0], [0, 0, 3]],
 [[0.0, 0.0, 0.0],
  [0.5, 0.0, 0.5],
  [0.0, 0.5, 0.0],
  [0.5, 0.5, 0.5],
  [0.3, 0.0, 0.0],
  [0.7, 0.0, 0.0],
  [0.2, 0.3, 0.5],
  [0.3, 0.5, 0.0],
  [0.7, 0.5, 0.0],
  [0.2, 0.8, 0.5],
  [0.8, 0.2, 0.5],
  [0.8, 0.7, 0.5]],
 [50, 50, 50, 50, 8, 8, 8, 8, 8, 8, 8, 8])

In [12]:
jack = 2
struct_list = list()
parent_lattice = unko.get_superlattice(rutile, reduced_HNF[jack], 2)
o_sublattice = unko.get_o_superlattice(rutile, reduced_HNF[jack], 2)
parent_lattice_jun = permutation.generate_per(parent_lattice, o_sublattice)
for ken in range(8):
    can = permutation.make_candidate(o_sublattice, ken)
    auncko = permutation.unique(o_sublattice, parent_lattice_jun, can)
    struct_list.extend(auncko)
print(struct_list)

['00000001', '00000011', '00000110', '00001001', '00000111', '00001011', '00010011', '00011001', '00001111', '00010111', '00011011', '00011110', '00101011', '00101101', '00110101', '00011111', '00101111', '00110111', '01100111', '00111111', '01101111', '01111110', '01111111']


In [19]:
auncko = ['00000001']
mapping_color_species = [DummySpecie('X'), Specie('O'), Specie('Sn')]
index = 2
pstruct = Structure(parent_lattice[0], parent_lattice[2], parent_lattice[1])
for k in range(len(struct_list)):
    for i in range(8):
        if int(auncko[0][i]) == 0:
            pstruct[2*index+i] = mapping_color_species[0]

pstruct

Structure Summary
Lattice
    abc : 5.0 5.0 6.0
 angles : 53.13010235415598 53.13010235415598 68.89980397590698
 volume : 96.0
      A : 4.0 0.0 3.0
      B : 0.0 4.0 3.0
      C : 0.0 0.0 6.0
PeriodicSite: Sn (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Sn (0.0000, 0.0000, 3.0000) [0.0000, 0.0000, 0.5000]
PeriodicSite: Sn (2.0000, 2.0000, 4.5000) [0.5000, 0.5000, 0.2500]
PeriodicSite: Sn (2.0000, 2.0000, 7.5000) [0.5000, 0.5000, 0.7500]
PeriodicSite: X0+ (1.2000, 1.2000, 3.0000) [0.3000, 0.3000, 0.2000]
PeriodicSite: X0+ (0.8000, 3.2000, 4.5000) [0.2000, 0.8000, 0.2500]
PeriodicSite: X0+ (3.2000, 0.8000, 4.5000) [0.8000, 0.2000, 0.2500]
PeriodicSite: X0+ (1.2000, 1.2000, 6.0000) [0.3000, 0.3000, 0.7000]
PeriodicSite: X0+ (2.8000, 2.8000, 6.0000) [0.7000, 0.7000, 0.3000]
PeriodicSite: X0+ (0.8000, 3.2000, 7.5000) [0.2000, 0.8000, 0.7500]
PeriodicSite: X0+ (3.2000, 0.8000, 7.5000) [0.8000, 0.2000, 0.7500]
PeriodicSite: O (2.8000, 2.8000, 9.0000) [0.7000, 0.7000, 0.800

In [24]:
for i in range(len(struct_list)):
    pstruct = Structure(parent_lattice[0], parent_lattice[2], parent_lattice[1])
    for j in range(8):
        if int(struct_list[i][j]) == 0:
            pstruct[2*index+j] = mapping_color_species[0]
    pstruct.remove_species([mapping_color_species[0], ])
    name = 'SnOx_index={}'.format(index)
    filename = os.path.join('/home/asa/python/homework', name, "SnOx_index={}_{}.cif".format(index, i))
    write_cif(filename, pstruct, refine_cell=True)


In [17]:
struct_list[0][7]

'1'

In [21]:
pstruct = Structure(parent_lattice[0], parent_lattice[2], parent_lattice[1])
pstruct

Structure Summary
Lattice
    abc : 5.0 5.0 6.0
 angles : 53.13010235415598 53.13010235415598 68.89980397590698
 volume : 96.0
      A : 4.0 0.0 3.0
      B : 0.0 4.0 3.0
      C : 0.0 0.0 6.0
PeriodicSite: Sn (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Sn (0.0000, 0.0000, 3.0000) [0.0000, 0.0000, 0.5000]
PeriodicSite: Sn (2.0000, 2.0000, 4.5000) [0.5000, 0.5000, 0.2500]
PeriodicSite: Sn (2.0000, 2.0000, 7.5000) [0.5000, 0.5000, 0.7500]
PeriodicSite: O (1.2000, 1.2000, 3.0000) [0.3000, 0.3000, 0.2000]
PeriodicSite: O (0.8000, 3.2000, 4.5000) [0.2000, 0.8000, 0.2500]
PeriodicSite: O (3.2000, 0.8000, 4.5000) [0.8000, 0.2000, 0.2500]
PeriodicSite: O (1.2000, 1.2000, 6.0000) [0.3000, 0.3000, 0.7000]
PeriodicSite: O (2.8000, 2.8000, 6.0000) [0.7000, 0.7000, 0.3000]
PeriodicSite: O (0.8000, 3.2000, 7.5000) [0.2000, 0.8000, 0.7500]
PeriodicSite: O (3.2000, 0.8000, 7.5000) [0.8000, 0.2000, 0.7500]
PeriodicSite: O (2.8000, 2.8000, 9.0000) [0.7000, 0.7000, 0.8000]

In [23]:
index

2

In [ ]:
index = 2
name = 'SnOx_index={}'.format(index)

i = "not_refined"
filename = os.path.join('/home/asa/python/homework', name, "SnOx_index={}_{}.cif".format(index, i))
write_cif(filename, pstruct, refine_cell=False)

In [ ]:
index = 2
name = 'SnOx_index={}'.format(index)

i = "not_refined"
filename = os.path.join('/home/asa/python/homework', name, "SnOx_index={}_{}.cif".format(index, i))
write_cif(filename, pstruct, refine_cell=False)